In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
from census import Census
import statsmodels.api as sm
from math import exp

In [2]:
pd.set_option('display.max_columns', 80)

In [3]:
c = Census('you_key_here')

## Hammer - Data Collection

In [4]:
#Read in historic data collected by Hauer in previous paper and avaliable in GitHub Repo
historic = pd.read_excel('./Hauer_DATA/historicaldat.xlsx')


In [5]:
# download.file("https://www2.census.gov/geo/docs/reference/codes/national_county.txt", "DATA/national_county.txt")
fips = pd.read_csv('./Hauer_DATA/national_county.txt')
# Filter out PR, Guam, etc
fips = fips[fips["State ANSI"]<=56]

In [6]:
# Downloading the county-level population projections
# download.file("https://osf.io/uh5sj/download/", destfile = "DATA/SSP_asrc.zip", mode = 'wb')
# unzip(zipfile='DATA/SSP_asrc.zip', exdir = "DATA") 
# ^^^^ R Code
#
controls = pd.read_csv("./Hauer_DATA/SSP_asrc.csv")
controls.groupby(['YEAR','STATE','COUNTY','GEOID']).sum()
contros=controls[controls['GEOID'].isin(fips)]
#controls[controls['STATE']==12]

/Users/jt/Environments/gis_projects/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
#Getting the Decennial Data - Note state fips set to 12 = FL & county set to 57 = Hillsborough
data_decen = pd.DataFrame(c.sf1.get(fields=['GEO_ID',"H001001", "P001001", "P043001"], geo={'for':"block group:*",
                                                        'in':'state:12 county:057 tract:*'}, year=2010))
#Change data type of populaiton from string to float
data_decen['P001001'] = data_decen['P001001'].astype(float)

#data_decen = data_decen.groupby('GEO_ID').sum()
#data_decen['GEO_ID'] = data_decen['GEO_ID'].apply(lambda x: str(x)[0:12])




In [11]:
#grp_decen = data_decen.groupby('GEO_ID').sum()
#Estimate persons per household as the Total pop minus GQ divided by the # of HU's
data_decen['pphu']= (data_decen['P001001']-data_decen['P043001'])/data_decen['H001001']
#data_decen.reset_index(inplace=True)
data_decen['fips']=data_decen['GEO_ID'].apply(lambda x: str(x)[9:14])
data_decen

,GEO_ID,H001001,P001001,P043001,state,county,tract,block group,pphu,fips
0,1500000US120570001011,1612.0,3106.0,0.0,12,057,000101,1,1.926799,12057
1,1500000US120570001012,471.0,1202.0,0.0,12,057,000101,2,2.552017,12057
2,1500000US120570001021,441.0,981.0,0.0,12,057,000102,1,2.224490,12057
3,1500000US120570001022,512.0,1364.0,1.0,12,057,000102,2,2.662109,12057
4,1500000US120570001023,489.0,858.0,7.0,12,057,000102,3,1.740286,12057
...,...,...,...,...,...,...,...,...,...,...
876,1500000US120579805001,3.0,15.0,0.0,12,057,980500,1,5.000000,12057
877,1500000US120579806001,0.0,0.0,0.0,12,057,980600,1,NaN,12057
878,1500000US120579807001,1.0,5.0,0.0,12,057,980700,1,5.000000,12057
879,1500000US120579900000,0.0,0.0,0.0,12,057,990000,0,NaN,12057


In [12]:
data_decen.to_csv('census2010dat.csv')

In [13]:
var_list = ["GEO_ID","NAME",
            'B25034_010E',
            'B25034_009E',
            'B25034_008E',
            'B25034_007E',
            'B25034_006E',
            'B25034_005E',
            'B25034_004E',
            'B25034_003E',
            "B25034_002E",
            'B25034_001E'] 
col_names = ["GEO_ID",'NAME', 
             'a1939', 
             'a1949', 
             'a1959', 
             'a1969', 
             'a1979', 
             'a1989', 
             'a1999', 
             'a2009',
             'a2019',
             'Totala',
             'state','county','tract','block group']
# Get ACS data
dat_acs = c.acs5.get(fields=var_list, geo={'for':"block group:*",'in':'state:12 county:057 tract:*'}, year=2013)
dat_acs =pd.DataFrame(dat_acs)
dat_acs.columns = col_names
dat_acs['fips'] = dat_acs['GEO_ID'].apply(lambda x: str(x)[9:14])
dat_acs = dat_acs[['GEO_ID', 'NAME', 'state', 'county', 'tract','block group', 'fips',
       'a1939', 'a1949', 'a1959', 'a1969', 'a1979', 'a1989',
       'a1999', 'a2009', 'a2019', 'Totala']]

In [ ]:
#dat_acs

In [ ]:
#dat_acs.to_csv('ACS2019.csv',index=False)

## Hammer - Calculation

In [14]:
#Isolate state and county fips from GEO_ID to get combined fips code

dat_acs = pd.read_csv('ACS2019.csv')
dat_acs['fips'] = dat_acs['GEO_ID'].apply(lambda x: str(x)[9:14])
dat_acs['fips'] = dat_acs['fips'].astype(int)
#Group by and sum by fips code
dat_acs = dat_acs.groupby('fips').sum() 

# reset index to bring fips column back into dataframs 
dat_acs.reset_index(inplace=True)

dat_acs.columns = ['fips', 'state', 'county', 'tract', 'block group', 't1939', 't1949',
       't1959', 't1969', 't1979', 't1989', 't1999', 't2009', 't2019', 'Totala']
#join acs data with historic
data_sum = dat_acs.merge(historic, how="left", left_on="fips",right_on='FIPS')

In [15]:
#Creat adjustment factors for each decade
data_sum['adj_1940'] = data_sum['h1940']/data_sum['t1939']
data_sum['adj_1950'] = data_sum['h1950']/(data_sum['t1939']+data_sum['t1949'])
data_sum['adj_1960'] = data_sum['h1960']/(data_sum['t1939']+data_sum['t1949']+data_sum['t1959'])
data_sum['adj_1970'] = data_sum['h1970']/(data_sum['t1939']+data_sum['t1949']+data_sum['t1959']+data_sum['t1969'])
data_sum['adj_1980'] = data_sum['h1980']/(data_sum['t1939']+data_sum['t1949']+data_sum['t1959']+data_sum['t1969']+data_sum['t1979'])
data_sum['adj_1990'] = data_sum['h1990']/(data_sum['t1939']+data_sum['t1949']+data_sum['t1959']+data_sum['t1969']+data_sum['t1979']+data_sum['t1989'])
data_sum['adj_2000'] = data_sum['h2000']/(data_sum['t1939']+data_sum['t1949']+data_sum['t1959']+data_sum['t1969']+data_sum['t1979']+data_sum['t1989']+data_sum['t1999'])
data_sum['adj_2010'] = data_sum['h2010']/(data_sum['t1939']+data_sum['t1949']+data_sum['t1959']+data_sum['t1969']+data_sum['t1979']+data_sum['t1989']+data_sum['t1999']+data_sum['t2009'])

In [16]:
#Read in original non manipulated acs data from before 
dat_acs_orig = pd.read_csv('ACS2019.csv')
#Merge manipulated acs data with original data
joined = data_sum.merge(dat_acs_orig,how='right',on='fips')

In [17]:
#Creates number of housing units using the Hammer Method
joined['hat_1940'] = (joined['h1940']/(joined['t1939'])) * joined['a1939']
joined['hat_1950'] = (joined['h1950']/(joined['t1939']+joined['t1949'])) * (joined['a1939']+joined['a1949'])
joined['hat_1960'] = (joined['h1960']/(joined['t1939']+joined['t1949']+joined['t1959'])) * (joined['a1939']+joined['a1949']+joined['a1959'])
joined['hat_1970'] = (joined['h1970']/(joined['t1939']+joined['t1949']+joined['t1959']+joined['t1969'])) * (joined['a1939']+joined['a1949']+joined['a1959']+joined['a1969'])
joined['hat_1980'] = (joined['h1980']/(joined['t1939']+joined['t1949']+joined['t1959']+joined['t1969']+joined['t1979'])) * (joined['a1939']+joined['a1949']+joined['a1959']+joined['a1969']+joined['a1979'])
joined['hat_1990'] = (joined['h1990']/(joined['t1939']+joined['t1949']+joined['t1959']+joined['t1969']+joined['t1979']+joined['t1989'])) * (joined['a1939']+joined['a1949']+joined['a1959']+joined['a1969']+joined['a1979']+joined['a1989'])
joined['hat_2000'] = (joined['h2000']/(joined['t1939']+joined['t1949']+joined['t1959']+joined['t1969']+joined['t1979']+joined['t1989']+joined['t1999'])) * (joined['a1939']+joined['a1949']+joined['a1959']+joined['a1969']+joined['a1979']+joined['a1989']+joined['a1999'])
joined['hat_2010'] = (joined['h2010']/(joined['t1939']+joined['t1949']+joined['t1959']+joined['t1969']+joined['t1979']+joined['t1989']+joined['t1999']+joined['t2009'])) * (joined['a1939']+joined['a1949']+joined['a1959']+joined['a1969']+joined['a1979']+joined['a1989']+joined['a1999']+joined['a2009'])

In [18]:
#Add column with label for if the number of housing units grew or declined between 1940 and 2010
joined['GROWTH'] = np.where(joined['hat_2010']>=joined['hat_1940'], 'grow','decline')


In [19]:
#Select desired columns
joined = joined[['GEO_ID','fips','hat_1940','hat_1950',
                'hat_1960','hat_1970','hat_1980','hat_1990',
                'hat_2000','hat_2010','GROWTH']]
#Change fips column in data_decen to integer
data_decen['fips']=data_decen['fips'].astype(int)
#Merge joined and data_decen
joined = joined.merge(data_decen,how='left',on='GEO_ID')
#Select desired columns again
joined = joined[['GEO_ID', 'fips_x', 'hat_1940', 'hat_1950', 
      'hat_1960', 'hat_1970','hat_1980', 'hat_1990', 
      'hat_2000', 'hat_2010', 'GROWTH','pphu','P043001', 
      'H001001']]

In [20]:
#Convert joined df into a long format for modeling
joinedt = joined.melt(id_vars=['GEO_ID','fips_x','GROWTH','pphu','P043001','H001001'],var_name='YEAR',value_name='HU')
joinedt['YEAR']=joinedt['YEAR'].apply(lambda x: x[-4:])
joinedt['YEAR'] = joinedt['YEAR'].astype(int)
joinedt['GEO_ID'] = joinedt['GEO_ID'].apply(lambda x: x[9:])
joinedt['GEO_ID'] = joinedt['GEO_ID'].astype(int)

In [21]:
#Read in block groups form ArcGIS analysis
bg_df = pd.read_excel('./Data/7_11_2021_Exports/BlockGroupExport.xlsx')
slr_bgs = bg_df["GEOID"].unique()
#Filter joinedt for only the block groups with SLR impacts
joinedt= joinedt[joinedt['GEO_ID'].isin(slr_bgs)]

In [22]:
#model_df = pd.DataFrame(columns=['GEO_ID','const','YEAR'])
collect_list = []
for geo_id in joinedt.GEO_ID.unique():
    tempdf = joinedt[joinedt['GEO_ID']==geo_id]
    X = sm.add_constant(tempdf['YEAR'])
    y = tempdf['HU']
    
    model=sm.OLS(y, X)
    results = model.fit()
    
    dic = {"GEO_ID":geo_id,"const":results.params[0],"yr_coef":results.params[1]}
    #print(dic)
    collect_list.append(dic)
    #print(geo_id,"\n",results.params)
fitted_models = pd.DataFrame(collect_list)

In [23]:
joined['GEO_ID'] = joined['GEO_ID'].apply(lambda x: x[9:])
joined['GEO_ID'] = joined['GEO_ID'].astype(int)
fitted_models = fitted_models.merge(joined,how='left',on='GEO_ID')

In [24]:
fitted_models

,GEO_ID,const,yr_coef,fips_x,hat_1940,hat_1950,hat_1960,hat_1970,hat_1980,hat_1990,hat_2000,hat_2010,GROWTH,pphu,P043001,H001001
0,120570059006,-9442.631071,4.883217,12057,0.000000,0.000000,105.372828,302.377049,316.902580,308.134391,293.946936,287.051091,grow,2.612782,0.0,266.0
1,120570061013,9559.435823,-4.664300,12057,593.241765,509.891295,352.520007,271.721311,277.289758,269.042715,255.918836,249.915110,decline,2.589431,1.0,246.0
2,120570061014,24312.896712,-11.738654,12057,1660.432114,1427.141398,1354.519809,1000.491803,959.652572,931.301705,841.757136,857.138571,decline,1.452381,0.0,672.0
3,120570061034,-8029.269865,4.142595,12057,0.000000,49.880670,34.485653,149.098361,233.843436,251.796387,252.835477,246.904085,grow,1.103896,0.0,308.0
4,120570061032,-5007.006218,2.668976,12057,254.707062,218.920719,151.353699,135.163934,284.956756,347.226068,334.030610,387.418605,grow,1.409214,0.0,369.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,120570058003,-5071.938777,2.746603,12057,0.000000,346.393543,547.938706,408.278689,397.406058,357.573865,360.753058,402.473732,grow,2.222222,1.0,486.0
58,120570069001,-11801.150125,6.123954,12057,38.689680,113.617082,302.707397,321.885246,295.179420,324.230964,362.808631,590.160984,grow,2.253067,0.0,652.0
59,120570070022,-9081.535509,4.696978,12057,148.310441,127.472824,105.372828,76.639344,126.505466,113.825764,243.585398,618.263888,grow,2.198425,0.0,635.0
60,120570070013,-18169.466273,9.408230,12057,0.000000,0.000000,390.837399,493.278689,585.247507,650.761438,594.060592,580.124232,grow,2.239777,0.0,538.0


In [25]:
#Filter into two different datasets for declining or growing pop for different extrapolaiton form
decline_models = fitted_models[fitted_models['GROWTH']=="decline"].copy()
grow_models = fitted_models[fitted_models['GROWTH']=="grow"].copy()

In [26]:
decline_models['h2010'] = np.exp((decline_models['yr_coef']*2010 + decline_models['const']))
decline_models['adj_factor'] = decline_models['hat_2010']-decline_models['h2010']

decline_models['HU_2020']= (np.exp(decline_models['yr_coef']*2020 )+ decline_models['const'])+decline_models['adj_factor']
decline_models['HU_2030']= (np.exp(decline_models['yr_coef']*2030 )+ decline_models['const'])+decline_models['adj_factor']
decline_models['HU_2040']= (np.exp(decline_models['yr_coef']*2040 )+ decline_models['const'])+decline_models['adj_factor']
decline_models['HU_2050']= (np.exp(decline_models['yr_coef']*2050 )+ decline_models['const'])+decline_models['adj_factor']
decline_models['HU_2060']= (np.exp(decline_models['yr_coef']*2060 )+ decline_models['const'])+decline_models['adj_factor']
decline_models['HU_2070']= (np.exp(decline_models['yr_coef']*2070 )+ decline_models['const'])+decline_models['adj_factor']

/Users/jt/Environments/gis_projects/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [27]:
#Calculate projected population using HU method
decline_models['pop_2020'] = decline_models['HU_2020']*decline_models['pphu'] + decline_models['P043001']
decline_models['pop_2030'] = decline_models['HU_2030']*decline_models['pphu'] + decline_models['P043001']
decline_models['pop_2040'] = decline_models['HU_2040']*decline_models['pphu'] + decline_models['P043001']
decline_models['pop_2050'] = decline_models['HU_2050']*decline_models['pphu'] + decline_models['P043001']
decline_models['pop_2060'] = decline_models['HU_2060']*decline_models['pphu'] + decline_models['P043001']
decline_models['pop_2070'] = decline_models['HU_2070']*decline_models['pphu'] + decline_models['P043001']

In [28]:
decline_models

,GEO_ID,const,yr_coef,fips_x,hat_1940,hat_1950,hat_1960,hat_1970,hat_1980,hat_1990,hat_2000,hat_2010,GROWTH,pphu,P043001,H001001,h2010,adj_factor,HU_2020,HU_2030,HU_2040,HU_2050,HU_2060,HU_2070,pop_2020,pop_2030,pop_2040,pop_2050,pop_2060,pop_2070
1,120570061013,9559.435823,-4.664300,12057,593.241765,509.891295,352.520007,271.721311,277.289758,269.042715,255.918836,249.915110,decline,2.589431,1.0,246.0,9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-2.551592e+80,-2.551592e+80,-2.551592e+80,-2.551592e+80,-2.551592e+80,-2.551592e+80
2,120570061014,24312.896712,-11.738654,12057,1660.432114,1427.141398,1354.519809,1000.491803,959.652572,931.301705,841.757136,857.138571,decline,1.452381,0.0,672.0,inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf
32,120570067002,6199.768036,-2.852616,12057,702.862526,759.294647,524.948271,475.163934,527.745023,489.795712,477.920718,569.083806,decline,1.777391,0.0,575.0,2.430477e+202,-2.430477e+202,-2.430477e+202,-2.430477e+202,-2.430477e+202,-2.430477e+202,-2.430477e+202,-2.430477e+202,-4.319909e+202,-4.319909e+202,-4.319909e+202,-4.319909e+202,-4.319909e+202,-4.319909e+202


In [29]:
grow_models['h2010'] = ((grow_models['yr_coef']*2010 + grow_models['const']))
grow_models['adj_factor'] = grow_models['hat_2010']-grow_models['h2010']
#Calculate projected housing units for each year
grow_models['HU_2020']= ((grow_models['yr_coef']*2020 )+ grow_models['const'])+grow_models['adj_factor']
grow_models['HU_2030']= ((grow_models['yr_coef']*2030 )+ grow_models['const'])+grow_models['adj_factor']
grow_models['HU_2040']= ((grow_models['yr_coef']*2040 )+ grow_models['const'])+grow_models['adj_factor']
grow_models['HU_2050']= ((grow_models['yr_coef']*2050 )+ grow_models['const'])+grow_models['adj_factor']
grow_models['HU_2060']= ((grow_models['yr_coef']*2060 )+ grow_models['const'])+grow_models['adj_factor']
grow_models['HU_2070']= ((grow_models['yr_coef']*2070 )+ grow_models['const'])+grow_models['adj_factor']

In [30]:
#Calculate projected population using HU method
grow_models['pop_2020'] = grow_models['HU_2020']*grow_models['pphu'] + grow_models['P043001']
grow_models['pop_2030'] = grow_models['HU_2030']*grow_models['pphu'] + grow_models['P043001']
grow_models['pop_2040'] = grow_models['HU_2040']*grow_models['pphu'] + grow_models['P043001']
grow_models['pop_2050'] = grow_models['HU_2050']*grow_models['pphu'] + grow_models['P043001']
grow_models['pop_2060'] = grow_models['HU_2060']*grow_models['pphu'] + grow_models['P043001']
grow_models['pop_2070'] = grow_models['HU_2070']*grow_models['pphu'] + grow_models['P043001']

In [32]:
grow_models.append(decline_models)

,GEO_ID,const,yr_coef,fips_x,hat_1940,hat_1950,hat_1960,hat_1970,hat_1980,hat_1990,hat_2000,hat_2010,GROWTH,pphu,P043001,H001001,h2010,adj_factor,HU_2020,HU_2030,HU_2040,HU_2050,HU_2060,HU_2070,pop_2020,pop_2030,pop_2040,pop_2050,pop_2060,pop_2070
0,120570059006,-9442.631071,4.883217,12057,0.000000,0.000000,105.372828,302.377049,316.902580,308.134391,293.946936,287.051091,grow,2.612782,0.0,266.0,3.726357e+02,-8.558462e+01,3.358833e+02,3.847154e+02,4.335476e+02,4.823798e+02,5.312120e+02,5.800441e+02,8.775897e+02,1.005178e+03,1.132765e+03,1.260353e+03,1.387941e+03,1.515529e+03
3,120570061034,-8029.269865,4.142595,12057,0.000000,49.880670,34.485653,149.098361,233.843436,251.796387,252.835477,246.904085,grow,1.103896,0.0,308.0,2.973463e+02,-5.044225e+01,2.883300e+02,3.297560e+02,3.711819e+02,4.126079e+02,4.540338e+02,4.954598e+02,3.182864e+02,3.640163e+02,4.097463e+02,4.554762e+02,5.012062e+02,5.469361e+02
4,120570061032,-5007.006218,2.668976,12057,254.707062,218.920719,151.353699,135.163934,284.956756,347.226068,334.030610,387.418605,grow,1.409214,0.0,369.0,3.576364e+02,2.978225e+01,4.141084e+02,4.407981e+02,4.674879e+02,4.941777e+02,5.208674e+02,5.475572e+02,5.835673e+02,6.211789e+02,6.587905e+02,6.964021e+02,7.340137e+02,7.716253e+02
5,120570065012,-5759.248680,3.005004,12057,0.000000,0.000000,283.548701,231.311475,212.120276,223.052507,230.224174,224.823232,grow,2.014760,0.0,271.0,2.808102e+02,-5.598697e+01,2.548733e+02,2.849233e+02,3.149734e+02,3.450234e+02,3.750735e+02,4.051235e+02,5.135085e+02,5.740522e+02,6.345958e+02,6.951394e+02,7.556830e+02,8.162267e+02
6,120570065014,-9453.431670,4.924433,12057,0.000000,44.338374,348.688268,353.934426,324.569578,400.114807,357.669699,349.278949,grow,2.128028,47.0,289.0,4.446794e+02,-9.540048e+01,3.985233e+02,4.477676e+02,4.970120e+02,5.462563e+02,5.955006e+02,6.447450e+02,8.950686e+02,9.998619e+02,1.104655e+03,1.209448e+03,1.314242e+03,1.419035e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,120570070013,-18169.466273,9.408230,12057,0.000000,0.000000,390.837399,493.278689,585.247507,650.761438,594.060592,580.124232,grow,2.239777,0.0,538.0,7.410768e+02,-1.609526e+02,6.742065e+02,7.682888e+02,8.623711e+02,9.564534e+02,1.050536e+03,1.144618e+03,1.510072e+03,1.720796e+03,1.931519e+03,2.142242e+03,2.352966e+03,2.563689e+03
61,120570063001,-6428.120670,3.429448,12057,45.137960,299.284021,467.472183,352.540984,383.349896,410.462603,390.558867,411.506808,grow,2.272727,0.0,385.0,4.650698e+02,-5.356304e+01,4.458013e+02,4.800958e+02,5.143902e+02,5.486847e+02,5.829792e+02,6.172737e+02,1.013185e+03,1.091127e+03,1.169069e+03,1.247011e+03,1.324953e+03,1.402895e+03
1,120570061013,9559.435823,-4.664300,12057,593.241765,509.891295,352.520007,271.721311,277.289758,269.042715,255.918836,249.915110,decline,2.589431,1.0,246.0,9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-9.853873e+79,-2.551592e+80,-2.551592e+80,-2.551592e+80,-2.551592e+80,-2.551592e+80,-2.551592e+80
2,120570061014,24312.896712,-11.738654,12057,1660.432114,1427.141398,1354.519809,1000.491803,959.652572,931.301705,841.757136,857.138571,decline,1.452381,0.0,672.0,inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf,-inf


In [35]:
grow_models = grow_models.sort_index()

In [37]:
grow_models.to_csv('housing_unit_and_populaiton_projections.csv',index=False)